In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import pytesseract
import cv2
import re
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def process_image(image_path):
    try:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
        
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(image, config=custom_config)
        
        values = extract_values_from_text(text)

        print("Values extracted from the image:")
        for key, value in values.items():
            if key not in ['WBC', 'RBC', 'Hemoglobin', 'Platelet']:
                print(key + ": " + value)

        result = compare_(values)
        print("Result:", result)
    except pytesseract.TesseractError as e:
        print("Tesseract Error:", e)
    except Exception as e:
        print("An error occurred:", e)

def is_cbc_analysis(text):
    cbc_keywords = ['CBC', 'Complete Blood Count']
    for keyword in cbc_keywords:
        if keyword in text:
            return True
    return False

def extract_values_from_text(text):
    values = {}
    lines = text.split('\n')
    for line in lines:
        if 'Platelet' in line:
            platelet_value = re.search(r"\d+\.\d+", line)
            values['Platelet'] = platelet_value.group() if platelet_value else None
        elif 'RBC' in line:
            rbc_value = re.search(r"\d+\.\d+", line)
            values['RBC'] = rbc_value.group() if rbc_value else None
        elif 'WBC' in line:
            wbc_value = re.search(r"\d+\.\d+", line)
            values['WBC'] = wbc_value.group() if wbc_value else None
        elif 'Hemoglobin' in line:
            hemoglobin_value = re.search(r"\d+\.\d+", line)
            values['Hemoglobin'] = hemoglobin_value.group() if hemoglobin_value else None
    return values

def compare_(values):
    platelet_min = 150
    platelet_max = 400
    rbc_min = 4.40
    rbc_max = 6.00
    wbc_min = 4.00
    wbc_max = 11.00
    hemoglobin_min = 13.5
    hemoglobin_max = 18.0
    
    platelet = float(values.get('Platelet', 0)) if values.get('Platelet') is not None else 0
    rbc = float(values.get('RBC', 0)) if values.get('RBC') is not None else 0
    wbc = float(values.get('WBC', 0)) if values.get('WBC') is not None else 0
    hemoglobin = float(values.get('Hemoglobin', 0)) if values.get('Hemoglobin') is not None else 0

    result = "NORMAL" if (platelet_min <= platelet <= platelet_max and
                          rbc_min <= rbc <= rbc_max and
                          wbc_min <= wbc <= wbc_max and
                          hemoglobin_min <= hemoglobin <= hemoglobin_max) else "ABNORMAL"
    
    if result == "NORMAL":
        print("Hemoglobin:", hemoglobin)
    return result

def upload_image():
    root = tk.Tk()
    root.withdraw() 
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=(("Image files", ".jpg;.jpeg;.png"), ("All files", ".*")))
    if file_path:
        print("Processing image...")
        process_image(file_path)
    else:
        print("No image selected.")

upload_image()


Processing image...
Values extracted from the image:
Result: ABNORMAL


In [7]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.1/38.6 MB 871.5 kB/s eta 0:00:45
   ---------------------------------------- 0.2/38.6 MB 1.1 MB/s eta 0:00:36
   ---------------------------------------- 0.2/38.6 MB 952.6 kB/s eta 0:00:41
   ---------------------------------------- 0.2/38.6 MB 919.0 kB/s eta 0:00:42
   ---------------------------------------- 0.3/38.6 MB 947.5 kB/s eta 0:00:41
   ---------------------------------------- 0.3/38.6 MB 1.0 MB/s eta 0:00:38
   ---------------------------------------- 0.4/38.6 MB 998.3 kB/s eta 0:00:39
   ---------------------------------------- 0.4/38.6 MB 1.0 MB/s eta 0:00:38
   ---